***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [9]:
import numpy as np
import chart_studio
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

<div class="alert alert-block alert-info">
<b>Load the stock historical data, transform in montly data and get percentage return</b>
</div>

In [2]:
# If working for any with a list like ['Stock name'....] know that the work flow is different
# R = pdr.DataReader(['FLRY3.SA'], 'yahoo', start = '2010/01/01', end = '2019/12/31')

R = data.DataReader('SNE', 'yahoo', start = '2017/01/01', end = '2019/01/31')
R = R.resample('M').last().pct_change()
R.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.025008,0.026246,0.025777,0.022795,-0.227816,0.022795
2017-03-31,0.084430,0.087407,0.086018,0.089470,-0.217370,0.092539
2017-04-30,0.023979,0.017267,0.018985,0.023421,0.509737,0.023421
2017-05-31,0.060133,0.066725,0.066376,0.060834,0.071867,0.060834


<div class="alert alert-block alert-info">
<b>Percentage return of the market index S&P 500 with GSPC</b>
</div>

In [3]:
Rm = data.DataReader('^GSPC', 'yahoo', start = '2017/01/01', end = '2019/01/31')
Rm = Rm.resample('M').last().pct_change()
Rm.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.038919,0.040468,0.040483,0.037198,0.030016,0.037198
2017-03-31,0.001081,0.001543,-0.000533,-0.000389,-0.203326,-0.000389
2017-04-30,0.009842,0.008364,0.012204,0.009091,0.108571,0.009091
2017-05-31,0.009320,0.008911,0.009170,0.011576,0.214573,0.011576


<div class="alert alert-block alert-info">
<b>Percentage return of risk free investment with DGS10 = One Month Treasury Constant Maturity Rate</b>
</div>

In [4]:
Rf = data.DataReader('DGS1MO', 'fred', start = '2017/01/01', end = '2019/01/31')
Rf = Rf.resample('M').last()
Rf = Rf/100
Rf.head()

,DGS1MO
DATE,
2017-01-31,0.0050
2017-02-28,0.0040
2017-03-31,0.0074
2017-04-30,0.0068
2017-05-31,0.0086


<div class="alert alert-block alert-info">
<b>Table for excess returns</b>
</div>

In [5]:
# When working with a list it's good to use concat method 
# CAPM = pd.concat([R['Adj Close'], Rm['Adj Close'], Rf['DGS1MO']], axis = 1)
# CAPM.rename(columns = {'FLRY3.SA': 'R', '^GSPC': 'Rm', 'DGS1MO': 'Rf'}, inplace = True)

df = pd.DataFrame({'R': R['Adj Close'], 'Rm': Rm['Adj Close'], 'Rf': Rf['DGS1MO']}).dropna()
df['R - Rf'] = df['R'] - df['Rf']
df['Rm - Rf'] = df['Rm'] - df['Rf']
df.head()

,R,Rm,Rf,R - Rf,Rm - Rf
Date,,,,,
2017-02-28,0.022795,0.037198,0.0040,0.018795,0.033198
2017-03-31,0.092539,-0.000389,0.0074,0.085139,-0.007789
2017-04-30,0.023421,0.009091,0.0068,0.016621,0.002291
2017-05-31,0.060834,0.011576,0.0086,0.052234,0.002976
2017-06-30,0.042873,0.004814,0.0084,0.034473,-0.003586


<div class="alert alert-block alert-info">
<b>Linear regression for excess returns</b>
</div>

In [6]:
x = df['Rm - Rf']
y = df['R - Rf']

x = sm.add_constant(x)
model = sm.OLS(y, x, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 R - Rf   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     10.58
Date:                Wed, 06 May 2020   Prob (F-statistic):            0.00365
Time:                        00:07:21   Log-Likelihood:                 35.989
No. Observations:                  24   AIC:                            -67.98
Df Residuals:                      22   BIC:                            -65.62
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0160      0.012      1.363      0.187      -0.008       0.040
Rm - Rf        1.0069      0.310      3.253      0.004       0.365       1.649
==============================================================================
Omnibus:                        0.841   Durbin-Watson:                   2.106
Prob(Omnibus):                  0.657   Jarque-Bera (JB):                0.861
Skew:                           0.348   Prob(JB):                        0.650
Kurtosis:                       2.387   Cond. No.                         26.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Add deviation from the mean, predicted values, simple residuals, squared residuals and standard error of estimate to table</b>
</div>

In [7]:
# Predicted values for excess return of stock(R - Rf)
y_pred = results.predict()
df['Predicted excess returns'] = y_pred

# Simple residuals
residual = results.resid
df['Residuals'] = results.resid

# Squared residuals
squared_res = df['Residuals']**2
df['Squared residuals'] =  squared_res

# Sum of squared residuals
sum_squared_res = df['Squared residuals'].sum()
print('Squared residual: %f' %sum_squared_res)

# Squared error of estimated
squared_error = results.bse[1]
print('Squared error of estimated: %f' %squared_error)

df.head()

Squared residual: 0.070024
Squared error of estimated: 0.309563


,R,Rm,Rf,R - Rf,Rm - Rf,Predicted excess returns,Residuals,Squared residuals
Date,,,,,,,,
2017-02-28,0.022795,0.037198,0.0040,0.018795,0.033198,0.049383,-0.030588,0.000936
2017-03-31,0.092539,-0.000389,0.0074,0.085139,-0.007789,0.008114,0.077025,0.005933
2017-04-30,0.023421,0.009091,0.0068,0.016621,0.002291,0.018263,-0.001642,0.000003
2017-05-31,0.060834,0.011576,0.0086,0.052234,0.002976,0.018953,0.033281,0.001108
2017-06-30,0.042873,0.004814,0.0084,0.034473,-0.003586,0.012346,0.022127,0.000490


<div class="alert alert-block alert-info">
<b>Excess monthly % returns plot</b>
</div>

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['Rm - Rf'], y = df['R - Rf'], 
    name = 'Observed excess returns', 
    mode = 'markers',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 255, 145)')))

fig.add_trace(go.Scatter(x = df['Rm - Rf'], y = df['Predicted excess returns'], 
    error_y = dict(value = squared_error*100, 
    color = 'rgb(255, 145, 255)'),
    name = 'Predicted excess returns',
    mode = 'lines',
    marker=dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(145, 255, 255)')))

fig.update_layout(title_text = 'Sony (SNE) and S&P 500 Index from 2017 to 2019', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(x = 0, y = 1.5),
    xaxis_title = 'Monthly market excess return (%)',
    yaxis_title = 'Monthly SNE excess return (%)',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel= dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Linear regression of monthly excess return - Sony (SNE) and S&P 500', auto_open = False)

$Linear\ Regression$
$$Y_i = b_0 + b_1 X_i + \epsilon_i$$ <br/>
$$(R\ -\ Rf)\ =\ b_{0}\ +\ b_{1}(Rm\ -\ Rf)\ +\ \epsilon$$